# Tool 2 - Structural Classifier (Async Pattern)

**Status:** ✅ Ready for Databricks | **LLM Cost:** ~$0.002 per run | **Performance:** ~10-15s

**Pattern:** Single async function with Pydantic AI classifier agent

**Showcase:** LLM-based data warehouse classification (FACT vs DIMENSION) + heuristic relationship detection.

**Key Features:**
- Single async `classify_structure()` function
- LLM-based FACT/DIMENSION classification with grain detection (transaction/event/snapshot/aggregate)
- Heuristic FK detection (column suffix matching: `product_id` → `products`)
- Size estimation (small/medium/large/huge) and SCD Type 2 detection
- Expected performance: ~10-15s per classification

**TODO:**
- [ ] Validate FK detection accuracy (compare with actual schema metadata)
- [ ] Add support for bridge tables (many-to-many relationships)
- [ ] Test with 50+ table schemas (current test: 10-20)
- [ ] Add confidence scores for classifications

**IDEA:**
- Use actual column metadata (names, types, nullability) instead of table-level heuristics
- Add ML-based FK detection (column name similarity + cardinality analysis)
- Support custom grain definitions (not just 4 predefined types)

In [ ]:
# Install dependencies
%pip install pydantic-ai>=0.0.49 pydantic>=2.8.0

In [ ]:
# Restart Python kernel to use new packages
dbutils.library.restartPython()  # type: ignore

In [ ]:
import asyncio
import json
import os
from datetime import datetime
from pydantic import BaseModel, Field
from pydantic_ai import Agent  # type: ignore

In [ ]:
# Configure Azure OpenAI from Databricks secrets
AZURE_ENDPOINT = dbutils.secrets.get(scope="mcop", key="azure-openai-endpoint").strip()  # type: ignore
AZURE_API_KEY = dbutils.secrets.get(scope="mcop", key="azure-openai-api-key").strip()  # type: ignore
DEPLOYMENT_NAME = dbutils.secrets.get(scope="mcop", key="azure-openai-deployment-name").strip()  # type: ignore

# Clean endpoint (remove /openai/v1/ if present - Pydantic AI will handle routing)
azure_endpoint_clean = AZURE_ENDPOINT.replace("/openai/v1/", "").replace("/openai/v1", "").rstrip("/")

# Set environment variables for Pydantic AI (Azure OpenAI compatible)
os.environ["OPENAI_BASE_URL"] = f"{azure_endpoint_clean}/openai/deployments/{DEPLOYMENT_NAME}"
os.environ["OPENAI_API_KEY"] = AZURE_API_KEY

MODEL_NAME = f"openai:{DEPLOYMENT_NAME}"
print(f"✅ Configured model: {MODEL_NAME}")
print(f"   Base URL: {os.environ['OPENAI_BASE_URL']}")


In [ ]:
# ==========================================
# PYDANTIC SCHEMAS - FACT/DIMENSION klasifikace
# ==========================================
# Tool 2 účel: Klasifikovat entity z Tool 0 jako FACT nebo DIMENSION tabulky
# podle data warehouse konceptů (Kimball dimensional modeling)

# ==========================================
# SCHEMA 1: FactTable (transakční/event data)
# ==========================================
# Co je FACT table:
# - Obsahuje měřitelná data (metrics, measures)
# - Vysoký počet řádků (transactional scale)
# - Časově závislá data (daily/hourly updates)
# - Cizí klíče na dimension tables
#
# Příklady:
# - orders (objednávky: amount, quantity, date)
# - clickstreams (web events: page_id, user_id, timestamp)
# - sensor_readings (IoT: temperature, pressure, device_id)
class FactTable(BaseModel):
    """Fact table (transakční/event data).

    Pydantic použití:
    - name: Název tabulky (str required)
    - entity_id: ID z Tool 1 mapping (propojení s business požadavkem)
    - grain: Granularita dat (transaction/event/snapshot/aggregate)
      * transaction: jednotlivé transakce (každý řádek = 1 objednávka)
      * event: události (každý řádek = 1 kliknutí)
      * snapshot: snímky stavu (každý den = 1 řádek per customer)
      * aggregate: agregace (každý měsíc = 1 řádek per product)
    - estimated_row_count: Velikostní odhad (small <1M, medium 1-10M, large 10-100M, huge >100M)
    """
    name: str = Field(description="Table name")
    entity_id: str = Field(description="Mapped entity ID from Tool 1")
    description: str = Field(default="", description="Table description")
    grain: str = Field(description="Granularity: transaction, event, snapshot, aggregate")
    estimated_row_count: str = Field(description="Size estimate: small/medium/large/huge")

# ==========================================
# SCHEMA 2: DimensionTable (referenční/master data)
# ==========================================
# Co je DIMENSION table:
# - Popisné atributy (descriptive data)
# - Nízký počet řádků (reference scale)
# - Relativně statická data (weekly/monthly updates)
# - Primární klíče (pro FK z fact tables)
#
# Příklady:
# - products (katalog produktů: name, category, price)
# - customers (klienti: name, email, segment)
# - locations (lokace: city, country, region)
class DimensionTable(BaseModel):
    """Dimension table (referenční/master data).

    Pydantic použití:
    - type: Typ dimension tabulky
      * master: Hlavní master data (products, customers)
      * reference: Číselníky (countries, categories)
      * lookup: Drobné lookup tables (status codes, types)
      * bridge: Propojovací tabulky (many-to-many relationships)
    - slowly_changing: SCD Type 2 flag (zda se trackují historické změny)
      * True: Pokud tabulka má history (např. customer měnil adresu)
      * False: Pokud tabulka nemá history (např. country list)
    """
    name: str = Field(description="Table name")
    entity_id: str = Field(description="Mapped entity ID from Tool 1")
    description: str = Field(default="", description="Table description")
    type: str = Field(description="Type: master, reference, lookup, bridge")
    slowly_changing: bool = Field(default=False, description="SCD Type 2?")

# ==========================================
# SCHEMA 3: Relationship (cizí klíč vztah)
# ==========================================
# Co je Relationship:
# - Propojení mezi FACT a DIMENSION (FK constraint)
# - Detekováno heuristikou (column suffix matching)
# - Confidence score pro kvalitu detekce
#
# Příklad:
# orders (fact) --(product_id)--> products (dimension)
# clickstreams (fact) --(user_id)--> users (dimension)
class Relationship(BaseModel):
    """Foreign key relationship mezi tabulkami.

    Pydantic použití:
    - from_table: Source tabulka (obvykle FACT)
    - to_table: Target tabulka (obvykle DIMENSION)
    - relationship_type: Kardinalita vztahu
      * one-to-one: 1 řádek v source → max 1 řádek v target
      * one-to-many: 1 řádek v target → N řádků v source (nejčastější)
      * many-to-many: N řádků v source → M řádků v target (potřebuje bridge table)
    - confidence: Jak jistě jsme FK detekovali (0.0 = guess, 1.0 = confirmed)
    """
    from_table: str = Field(description="Source table name")
    to_table: str = Field(description="Target table name")
    relationship_type: str = Field(description="one-to-one, one-to-many, many-to-many")
    confidence: float = Field(description="Detection confidence 0-1")

# ==========================================
# SCHEMA 4: StructuralClassification (ROOT MODEL pro classifier_agent)
# ==========================================
# Co je StructuralClassification:
# - ROOT MODEL = Hlavní výstup classifier_agent (result_type)
# - Obsahuje 2 pole: facts[] a dimensions[]
# - LLM musí vrátit OBOJÍ (ne jen facts nebo jen dimensions)
#
# Proč 2 pole:
# - Jasná separace FACT vs DIMENSION
# - Validace, že každá entita je klasifikována
# - Type-safe access (result.data.facts, result.data.dimensions)
class StructuralClassification(BaseModel):
    """Complete structural classification (ROOT MODEL pro classifier_agent).

    Pydantic použití:
    - facts: Seznam všech FACT tables (transactional data)
    - dimensions: Seznam všech DIMENSION tables (reference data)

    Agent workflow:
    1. classifier_agent dostane entities + metadata
    2. LLM klasifikuje každou entitu jako FACT nebo DIMENSION
    3. Pydantic AI validuje, že output má structure: {facts: [...], dimensions: [...]}
    4. result.data vrátí StructuralClassification object
    """
    facts: list[FactTable] = Field(description="Fact tables")
    dimensions: list[DimensionTable] = Field(description="Dimension tables")

# ==========================================
# SCHEMA 5: StructuralMetrics (summary statistiky)
# ==========================================
# Co je StructuralMetrics:
# - Metadata o výsledku klasifikace
# - Počty tabulek a vztahů
# - Timestamp pro audit trail
class StructuralMetrics(BaseModel):
    """Metrics about the structure (pro reporting a audit).

    Pydantic použití:
    - fact_count, dimension_count: Počet klasifikovaných tabulek
    - relationship_count: Počet detekovaných FK vztahů
    - classification_timestamp: ISO 8601 timestamp (pro tracking změn)
    """
    fact_count: int
    dimension_count: int
    relationship_count: int
    classification_timestamp: str

In [ ]:
# ==========================================
# PYDANTIC AI AGENT - classifier_agent
# ==========================================
# Účel: LLM agent pro FACT/DIMENSION klasifikaci
# Input: business entities + technical metadata
# Output: StructuralClassification (facts[] + dimensions[])
#
# Proč potřebujeme LLM:
# - Sémantická analýza (product vs orders - který je FACT?)
# - Kontext z business požadavku (goal, scope)
# - Metadata quality assessment (granularity, size estimate)
# - Human-like reasoning (LLM umí vysvětlit klasifikaci)

classifier_agent = Agent(
    MODEL_NAME,                               # "openai:test-gpt-5-mini"
    result_type=StructuralClassification,     # LLM MUSÍ vrátit {facts: [...], dimensions: [...]}
    system_prompt="""You are a data warehouse architect.

Classify tables as FACT or DIMENSION:
- FACT: Transactional/event data (orders, clickstreams, sensor readings)
  * High row count
  * Time-dependent
  * Contains metrics/measures
  * Foreign keys to dimensions

- DIMENSION: Reference/master data (products, customers, locations)
  * Lower row count
  * Relatively static
  * Descriptive attributes
  * Primary keys

For each table, determine:
- Fact grain (transaction/event/snapshot/aggregate)
- Dimension type (master/reference/lookup/bridge)
- Size estimate (small/medium/large/huge)
- SCD Type 2 (slowly changing dimension)

Be specific and data-driven."""
)
# System prompt pravidla:
# 1. Definuje FACT vs DIMENSION kritéria (row count, time-dependency, metrics)
# 2. Specifikuje co určit (grain, type, size, SCD)
# 3. Instruuje "be specific and data-driven" (vyhni se guesswork)
#
# LLM dostane:
# - System prompt (výše)
# - User prompt (entities + metadata JSON)
# - JSON Schema pro StructuralClassification (automaticky přidáno Pydantic AI)
#
# LLM vrátí:
# - JSON matching StructuralClassification schema
# - Pydantic AI validuje a vrátí type-safe object

print("✅ Classifier agent created")

In [ ]:
# ==========================================
# HEURISTIC FK DETECTION - detect_fk_relationships()
# ==========================================
# Proč heuristika (ne LLM):
# - FK detection je pattern matching problém
# - Rychlé (<1ms per table)
# - Deterministické (konzistentní výsledky)
# - Nepotřebuje sémantické porozumění
#
# Pattern matching pravidla:
# 1. Singularizace dimension names (products → product)
# 2. Hledání dimension name v fact name
#    Příklad: fact "order_items" obsahuje "product" → FK na products dimension
# 3. Confidence 0.7 (heuristika není 100% jistá)
#
# V produkci bychom použili:
# - Skutečné column names z metadata katalogu
# - FK suffix patterns (product_id, customer_key, location_fk)
# - Actual FK constraints z database schema

def detect_fk_relationships(facts: list[FactTable], dimensions: list[DimensionTable]) -> list[Relationship]:
    """Heuristic FK detection based on column suffix matching.

    Args:
        facts: Seznam FACT tables (z classifier_agent)
        dimensions: Seznam DIMENSION tables (z classifier_agent)

    Returns:
        list[Relationship]: Detekované FK vztahy s confidence score

    Algoritmus:
        1. Singularizuj dimension names (products → product)
        2. Pro každý fact, zkontroluj jestli obsahuje dimension name
        3. Pokud ano, přidej relationship (one-to-many) s confidence 0.7

    Příklad:
        facts = [FactTable(name="order_items", ...)]
        dimensions = [DimensionTable(name="products", ...), DimensionTable(name="customers", ...)]

        Detekce:
        - "order_items" obsahuje "product" → FK na products (confidence 0.7)
        - "order_items" neobsahuje "customer" → no FK detected

    Limitations:
        - Heuristika je náchylná na false positives (product_categories → products FK?)
        - V produkci použít skutečné column names z metadata
        - Lepší: analyze actual FK constraints from database schema
    """
    relationships = []
    dimension_names = {d.name.lower().rstrip('s') for d in dimensions}  # Singularize
    # Singularizace: products → product, categories → categorie (simple plural removal)

    for fact in facts:
        # Simulate column names (in real scenario, from metadata)
        # Example: orders fact might have product_id, customer_id columns
        # For demo, assume naming convention: <entity>_id, <entity>_key, <entity>_fk

        for dim in dimensions:
            dim_singular = dim.name.lower().rstrip('s')
            # Heuristic: if fact name contains dimension name, likely FK
            # Example: "order_items" contains "product" → FK to products
            if dim_singular in fact.name.lower():
                relationships.append(Relationship(
                    from_table=fact.name,           # Source (FACT table)
                    to_table=dim.name,              # Target (DIMENSION table)
                    relationship_type="one-to-many",  # Standard DW pattern (1 dimension → N facts)
                    confidence=0.7  # Heuristic confidence (not 100% certain without actual schema)
                ))

    return relationships

print("✅ FK detection function defined")

In [ ]:
# ==========================================
# ASYNC CLASSIFICATION FUNCTION - classify_structure()
# ==========================================
# Tool 2 hlavní workflow:
# 1. Vezmi business context (Tool 0) + entity mappings (Tool 1)
# 2. Zavolej LLM classifier_agent (FACT/DIMENSION klasifikace)
# 3. Detekuj FK relationships (heuristika)
# 4. Spočítej metrics (fact_count, dimension_count, relationship_count)
# 5. Vrať kompletní strukturu
#
# Proč async:
# - classifier_agent.run() je async (Pydantic AI pattern)
# - Umožňuje později přidat paralel processing (multiple classifications)

async def classify_structure(tool0_context: dict, tool1_mappings: dict, metadata: dict) -> dict:
    """Classify tables into facts and dimensions.

    Args:
        tool0_context: Business request context from Tool 0
            Obsahuje: goal, scope_in, entities[], metrics[], sources[]

        tool1_mappings: Entity mappings from Tool 1
            Obsahuje: ranking (top 10 candidates), mapping (1:1 entity→candidate)

        metadata: Technical metadata (Collibra/Unity Catalog)
            Obsahuje: table schemas, column definitions, statistics

    Returns:
        dict with keys:
        - facts: List of FactTable objects (as dicts)
        - dimensions: List of DimensionTable objects (as dicts)
        - relationships: List of Relationship objects (FK detections)
        - metrics: StructuralMetrics (counts + timestamp)

    Workflow:
        Step 1: Prepare LLM prompt (combine business + technical context)
        Step 2: Call classifier_agent (LLM classification)
        Step 3: Detect FK relationships (heuristic post-processing)
        Step 4: Calculate metrics (summary statistics)
        Step 5: Assemble final structure (convert Pydantic → dict)
    """

    # ==========================================
    # STEP 1: Prepare LLM prompt
    # ==========================================
    # LLM potřebuje kontext:
    # - Business goal/scope (proč klasifikujeme?)
    # - Entities (co klasifikujeme?)
    # - Mappings from Tool 1 (které kandidáty jsou relevantní?)
    # - Technical metadata sample (jak vypadají data?)

    entities = tool0_context.get("entities", [])
    mappings = tool1_mappings.get("mappings", [])

    prompt = f"""Classify these entities into FACT and DIMENSION tables:

Business Context:
- Goal: {tool0_context.get('goal', 'N/A')}
- Scope: {tool0_context.get('scope_in', 'N/A')}

Entities from Business Request:
{json.dumps(entities, indent=2)}

Mapped Candidates (Tool 1):
{json.dumps(mappings[:10], indent=2)}  # Top 10 to avoid token overflow

Technical Metadata Sample:
{json.dumps(list(metadata.items())[:5], indent=2)}

Classify each entity as FACT or DIMENSION with justification."""
    # Prompt strategie:
    # - Business context first (framework pro decision-making)
    # - Entities second (co klasifikovat)
    # - Mappings third (Tool 1 suggestions)
    # - Metadata last (technical details)
    # - Token limit: Top 10 mappings + 5 metadata samples (avoid overflow)

    # ==========================================
    # STEP 2: Call LLM classifier
    # ==========================================
    # Pydantic AI workflow:
    # 1. agent.run(prompt) zavolá LLM
    # 2. LLM vrátí JSON matching StructuralClassification schema
    # 3. Pydantic AI validuje JSON → Pydantic object
    # 4. result.data = StructuralClassification instance

    result = await classifier_agent.run(prompt)
    classified = result.data  # Type: StructuralClassification
    # classified.facts = list[FactTable]
    # classified.dimensions = list[DimensionTable]

    # ==========================================
    # STEP 3: Detect FK relationships
    # ==========================================
    # Post-processing: Heuristic FK detection
    # Input: classified facts + dimensions
    # Output: list[Relationship] (FK constraints)
    # Pattern: name matching (product in order_items → FK to products)

    relationships = detect_fk_relationships(classified.facts, classified.dimensions)

    # ==========================================
    # STEP 4: Calculate metrics
    # ==========================================
    # Summary statistics pro reporting a audit:
    # - Kolik facts/dimensions jsme našli?
    # - Kolik FK relationships jsme detekovali?
    # - Kdy proběhla klasifikace? (timestamp)

    metrics = StructuralMetrics(
        fact_count=len(classified.facts),
        dimension_count=len(classified.dimensions),
        relationship_count=len(relationships),
        classification_timestamp=datetime.now().isoformat()  # ISO 8601 format
    )

    # ==========================================
    # STEP 5: Assemble final structure
    # ==========================================
    # Convert Pydantic objects → dict (pro JSON serialization)
    # .model_dump() = Pydantic metoda (convert BaseModel → dict)
    # List comprehension: [obj.model_dump() for obj in list]

    return {
        "facts": [f.model_dump() for f in classified.facts],
        "dimensions": [d.model_dump() for d in classified.dimensions],
        "relationships": [r.model_dump() for r in relationships],
        "metrics": metrics.model_dump()
    }

print("✅ Async classification function defined")

In [ ]:
# Load input data from DBFS
tool0_path = "/dbfs/FileStore/mcop/tool0_samples/sample_business_request.json"
tool1_path = "/dbfs/FileStore/mcop/tool1/filtered_dataset.json"
metadata_path = "/dbfs/FileStore/mcop/metadata/BA-BS_Datamarts_metadata.json"

with open(tool0_path, "r") as f:
    tool0_context = json.load(f)

with open(tool1_path, "r") as f:
    tool1_mappings = json.load(f)

with open(metadata_path, "r") as f:
    metadata = json.load(f)

print(f"✅ Loaded Tool 0 context: {len(tool0_context.get('entities', []))} entities")
print(f"✅ Loaded Tool 1 mappings: {len(tool1_mappings.get('mappings', []))} mappings")
print(f"✅ Loaded metadata: {len(metadata)} items")

In [ ]:
# Run classification
structure = await classify_structure(tool0_context, tool1_mappings, metadata)

print(f"\n✅ Classification complete")
print(f"   Facts: {structure['metrics']['fact_count']}")
print(f"   Dimensions: {structure['metrics']['dimension_count']}")
print(f"   Relationships: {structure['metrics']['relationship_count']}")

In [ ]:
# Save results to DBFS
output_path = "/dbfs/FileStore/mcop/tool2/structure.json"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(output_path, "w") as f:
    json.dump(structure, f, indent=2)

print(f"✅ Structure saved: {output_path}")

In [ ]:
# Display sample results
print("\n" + "="*80)
print("STRUCTURAL CLASSIFICATION RESULTS")
print("="*80)

print(f"\n📊 Metrics:")
print(f"   Facts: {structure['metrics']['fact_count']}")
print(f"   Dimensions: {structure['metrics']['dimension_count']}")
print(f"   Relationships: {structure['metrics']['relationship_count']}")
print(f"   Timestamp: {structure['metrics']['classification_timestamp']}")

print(f"\n📦 Sample Facts (top 3):")
for fact in structure['facts'][:3]:
    print(f"   - {fact['name']} (grain: {fact['grain']}, size: {fact['estimated_row_count']})")

print(f"\n🗂️  Sample Dimensions (top 3):")
for dim in structure['dimensions'][:3]:
    print(f"   - {dim['name']} (type: {dim['type']}, SCD2: {dim['slowly_changing']})")

print(f"\n🔗 Sample Relationships (top 3):")
for rel in structure['relationships'][:3]:
    print(f"   - {rel['from_table']} → {rel['to_table']} ({rel['relationship_type']}, confidence: {rel['confidence']:.1%})")

print("="*80)